In [ ]:
import pandas as pd
def read_file(filepath):
    file=pd.read_csv(filepath)
    return file
    # lines=zip(file.HADM_ID,file.Label)
    # return lines

filepath = "/clinicalBERT/data/extended/discharge/train.csv"
df_train = read_file(filepath)
df_train = df_train[df_train.Label == 0].HADM_ID
negative_HADMIDs_train = [hadm_id for hadm_id in df_train]

filepath = "/clinicalBERT/data/extended/discharge/val.csv"
df_val = read_file(filepath)
df_val = df_val[df_val.Label == 0].HADM_ID
negative_HADMIDs_val = [hadm_id for hadm_id in df_val]
                        
filepath = "/clinicalBERT/data/extended/discharge/test.csv"
df_test = read_file(filepath)
df_test = df_test[df_test.Label == 0].HADM_ID
negative_HADMIDs_test = [hadm_id for hadm_id in df_test]

negative_HADMIDs = negative_HADMIDs_train + negative_HADMIDs_val + negative_HADMIDs_test
with open("/clinicalBERT/data/extended/discharge/negative_hadmids.txt", "w") as file:
    for hadm in negative_HADMIDs:
        file.write(str(hadm)+"\n")
       

In [2]:
import pandas as pd
import numpy as np

with open("/clinicalBERT/data/extended/discharge/negative_hadmids.txt","r") as file:
    lines = file.readlines()
    negative_HADMIDs = [line.strip("\n") for line in lines]
    
def read_precomputed_npy(input_file):
    """Reads a comma separated value file."""
    array = np.load(input_file, allow_pickle=True)
    return array

def df_to_dict_array(pandas_df):
    entries = []
    for index, row in pandas_df.iterrows():
        entry = {}
        entry.update({"HADM_ID": row["HADM_ID"], "clinicalbert_embedding": row["clinicalbert_embedding"]})
        entries.append(entry)
    return(entries)
    
    
# df_train = pd.read_csv("/clinicalBERT/data/extended/discharge/train.csv")
# df_val   = pd.read_csv("/clinicalBERT/data/extended/discharge/val.csv")
# df_test  = pd.read_csv("/clinicalBERT/data/extended/discharge/test.csv")
# df_total = pd.concat([df_train, df_val, df_test])

df_total = pd.read_csv("/clinicalBERT/data/extended_folds/discharge/fold0_text.csv")
array = read_precomputed_npy("/clinicalBERT/data/extended_folds/discharge/fold0_text_precomputed.npy")
for i in range(1,10):
    df_fold = pd.read_csv("/clinicalBERT/data/extended_folds/discharge/fold"+str(i)+"_text.csv")
    df_total = pd.concat([df_total, df_fold])
    array_fold = read_precomputed_npy("/clinicalBERT/data/extended_folds/discharge/fold" + str(i) + "_text_precomputed.npy")
    array = np.concatenate([array, array_fold])
                                 
precomputed_df = pd.DataFrame.from_dict(list(array))
                                 
positive_HADMIDs = df_total[df_total.Label == 1.0].HADM_ID
positive_HADMIDs = list(np.unique(positive_HADMIDs))
negative_HADMIDs = list(np.unique(negative_HADMIDs))
valid_HADMIDs = negative_HADMIDs + positive_HADMIDs

df_total_valid = df_total[df_total.HADM_ID.isin(valid_HADMIDs)]
df_total_valid_HADMID = df_total_valid.groupby(['HADM_ID'])['HADM_ID'].nth(0).reset_index(drop=True)
valid_labels = df_total_valid.groupby(['HADM_ID'])['Label'].agg(np.min).reset_index(drop=True)
# print(valid_labels)
# print(np.unique(df_total_valid.Label, return_counts=True))
    
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits = 5, shuffle=True)
i = 0
for train_index, test_index in skfold.split(X=df_total_valid_HADMID, y=valid_labels):
    print("Fold Number {}:\n".format(i))
    fold_HADM_IDs = df_total_valid_HADMID[test_index]
    fold_df = df_total_valid[df_total_valid.HADM_ID.isin(fold_HADM_IDs)].reset_index(drop=True)
    filename = '/clinicalBERT/data/extended/discharge/fold' + str(i) + '_text.csv'
    fold_df.to_csv(filename, columns=["SUBJECT_ID","HADM_ID","ADMITTIME","DAYS_NEXT_ADMIT","DAYS_PREV_ADMIT","DURATION","DIAG_ICD9","SMALL_DIAG_ICD9","DIAG_CCS","PROC_ICD9","SMALL_PROC_ICD9","PROC_CCS","NDC","CUI","Label","TEXT","NEXT_SMALL_DIAG_ICD9","NEXT_DIAG_CCS","NEXT_SMALL_PROC_ICD9","NEXT_PROC_CCS","NEXT_CUI"])
    fold_precomputed_df = precomputed_df[precomputed_df.HADM_ID.isin(fold_HADM_IDs)].reset_index(drop=True)
    fold_precomputed_entries = df_to_dict_array(fold_precomputed_df)
    fold_array = np.asarray(fold_precomputed_entries, dtype=object) 
    array_filepath = "/clinicalBERT/data/extended/discharge/fold" + str(i) + "_text_precomputed.npy"
    np.save(array_filepath, fold_array)
    i+=1
    

Fold Number 0:

Fold Number 1:

Fold Number 2:

Fold Number 3:

Fold Number 4:

